In [18]:
# bug fixing test piece

import os
import json
import pandas as pd

data = []


# Load the JSON file
with open("train/data-000000000021") as f:
# Extract the relevant features from each log entry
    for line in f:
        entry = json.loads(line)
        # Extract the relevant features from the entry and add them to the data list
        features = {'latency': entry['httpRequest']['latency'],
                    'requestSize': entry['httpRequest']['requestSize'],
                    'requestMethod': entry['httpRequest']['requestMethod'],
                    'responseSize': entry['httpRequest']['responseSize'],
                    'requestUrl': entry['httpRequest']['requestUrl'],
                    'status': entry['httpRequest']['status'],
                    'source_app': entry['labels']['source_app'],
                    'timestamp': entry['timestamp']}
        data.append(features)

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(data)
df.head

In [1]:
import os
import json
import pandas as pd

folder_path = './train'  # change this to your folder path
data = []
errors = 0

# Loop through all the files in the folder
for filename in os.listdir(folder_path):
    if ("data" not in filename):
        continue
    file_path = os.path.join(folder_path, filename)
    # Load the JSON file
    with open(file_path) as f:
        # Extract the relevant features from each log entry
        for line in f:
            try:
                entry = json.loads(line)
                # Extract the relevant features from the entry and add them to the data list
                features = {'latency': entry['httpRequest']['latency'],
                            'requestSize': entry['httpRequest']['requestSize'],
                            'requestMethod': entry['httpRequest']['requestMethod'],
                            'responseSize': entry['httpRequest']['responseSize'],
                            'requestUrl': entry['httpRequest']['requestUrl'],
                            'status': entry['httpRequest']['status'],
                            'source_app': entry['labels']['source_app'],
                            'timestamp': entry['timestamp']}
                data.append(features)
            except json.JSONDecodeError:
                # Handle JSONDecodeError and continue to the next line
                errors += 1
                # print(f"Error decoding JSON in file {file_path}: {line}")
                continue

# Convert the list of dictionaries into a DataFrame
print("Number of errors: " + str(errors))
df = pd.DataFrame(data)
df.head()

Number of errors: 18


,latency,requestSize,requestMethod,responseSize,requestUrl,status,source_app,timestamp
0,0.001479,712,POST,1778,http://currencyservice:7000/hipstershop.Curren...,200,frontend,2023-04-15 09:29:32.235081 UTC
1,0.002243,712,POST,1778,http://currencyservice:7000/hipstershop.Curren...,200,frontend,2023-04-15 09:29:27.826092 UTC
2,0.001678,717,POST,1626,http://currencyservice:7000/hipstershop.Curren...,200,frontend,2023-04-15 09:29:29.543643 UTC
3,0.001477,717,POST,1626,http://currencyservice:7000/hipstershop.Curren...,200,frontend,2023-04-15 09:29:32.243186 UTC
4,0.001591,712,POST,1778,http://currencyservice:7000/hipstershop.Curren...,200,frontend,2023-04-15 09:29:31.537701 UTC


In [2]:
print(df.columns)
print(len(df))

Index(['latency', 'requestSize', 'requestMethod', 'responseSize', 'requestUrl',
       'status', 'source_app', 'timestamp'],
      dtype='object')
218570


In [3]:
# set timestamp as the index
df.set_index('timestamp', inplace=True)
# Sort the DataFrame by the index 
df = df.sort_index()

dforiginal = df.copy()

#catergorize latency
df['latency'] = (df['latency'] < 0.1).astype(int)


In [4]:
df.head()


,latency,requestSize,requestMethod,responseSize,requestUrl,status,source_app
timestamp,,,,,,,
2023-04-15 08:26:58.215025 UTC,1,730,POST,1757,http://productcatalogservice:3550/hipstershop....,200,frontend
2023-04-15 08:26:58.216662 UTC,1,730,POST,1695,http://productcatalogservice:3550/hipstershop....,200,frontend
2023-04-15 08:26:58.354216 UTC,1,730,POST,1747,http://productcatalogservice:3550/hipstershop....,200,frontend
2023-04-15 08:26:58.356309 UTC,1,730,POST,1695,http://productcatalogservice:3550/hipstershop....,200,frontend
2023-04-15 08:26:58.361683 UTC,1,730,POST,1742,http://productcatalogservice:3550/hipstershop....,200,frontend


In [5]:
#create keys for unique rows

df['key'] = df.groupby(['latency', 'requestSize', 'requestMethod', 'responseSize',
       'requestUrl', 'status', 'source_app']).ngroup()

In [6]:
#latex

latex_table = df.head().style.to_latex()

# print latex table
print(latex_table)

\begin{tabular}{lrllllllr}
 & latency & requestSize & requestMethod & responseSize & requestUrl & status & source_app & key \\
timestamp &  &  &  &  &  &  &  &  \\
2023-04-15 08:26:58.215025 UTC & 1 & 730 & POST & 1757 & http://productcatalogservice:3550/hipstershop.ProductCatalogService/GetProduct & 200 & frontend & 231 \\
2023-04-15 08:26:58.216662 UTC & 1 & 730 & POST & 1695 & http://productcatalogservice:3550/hipstershop.ProductCatalogService/GetProduct & 200 & frontend & 217 \\
2023-04-15 08:26:58.354216 UTC & 1 & 730 & POST & 1747 & http://productcatalogservice:3550/hipstershop.ProductCatalogService/GetProduct & 200 & frontend & 229 \\
2023-04-15 08:26:58.356309 UTC & 1 & 730 & POST & 1695 & http://productcatalogservice:3550/hipstershop.ProductCatalogService/GetProduct & 200 & frontend & 217 \\
2023-04-15 08:26:58.361683 UTC & 1 & 730 & POST & 1742 & http://productcatalogservice:3550/hipstershop.ProductCatalogService/GetProduct & 200 & frontend & 227 \\
\end{tabular}



In [7]:
df=df[['key']]

df = pd.get_dummies(df, columns=['key'])

In [8]:
df.head()

,key_0,key_1,key_2,key_3,key_4,key_5,key_6,key_7,key_8,key_9,...,key_377,key_378,key_379,key_380,key_381,key_382,key_383,key_384,key_385,key_386
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-04-15 08:26:58.215025 UTC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-04-15 08:26:58.216662 UTC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-04-15 08:26:58.354216 UTC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-04-15 08:26:58.356309 UTC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-04-15 08:26:58.361683 UTC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
from sklearn.preprocessing import StandardScaler


train_size = int(len(df) * 0.95)
train = df[:train_size][df.columns]
test = df[train_size:][df.columns]

In [10]:
# lstm autoencoder to recreate a timeseries
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
'''
A UDF to convert input data into 3-D
array as required for LSTM network.
'''

def temporalize(X, y, lookback):
    output_X = []
    output_y = []
    for i in range(len(X)-lookback-1):
        t = []
        for j in range(1,lookback+1):
            # Gather past records upto the lookback period
            t.append(X[[(i+j+1)], :])
        output_X.append(t)
        output_y.append(y[i+lookback+1])
    return output_X, output_y

2023-05-04 17:01:03.890992: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [11]:
timeseries = np.array(df.values)
timeseries.shape

(218570, 387)

In [12]:
import numpy as np

timesteps = 10
X, y = temporalize(X = timeseries, y = np.zeros(len(timeseries)), lookback = timesteps)

n_features = timeseries.shape[1]
X = np.array(X)
X = X.reshape(X.shape[0], timesteps, n_features)

In [13]:
# define model
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(timesteps,n_features), return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=False))
model.add(RepeatVector(timesteps))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')
model.summary()

2023-05-04 17:02:20.993062: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 128)           264192    
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 repeat_vector (RepeatVector  (None, 10, 64)           0         
 )                                                               
                                                                 
 lstm_2 (LSTM)               (None, 10, 64)            33024     
                                                                 
 lstm_3 (LSTM)               (None, 10, 128)           98816     
                                                                 
 time_distributed (TimeDistr  (None, 10, 387)          49923     
 ibuted)                                                

In [24]:
summary = model.summary()
summary_df = pd.read_html(summary, header=0, index_col=0)[0]
# Convert the DataFrame to a LaTeX table and print it
summary_df.style.to_latex()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 10, 128)           264192    
                                                                 
 lstm_1 (LSTM)               (None, 64)                49408     
                                                                 
 repeat_vector (RepeatVector  (None, 10, 64)           0         
 )                                                               
                                                                 
 lstm_2 (LSTM)               (None, 10, 64)            33024     
                                                                 
 lstm_3 (LSTM)               (None, 10, 128)           98816     
                                                                 
 time_distributed (TimeDistr  (None, 10, 387)          49923     
 ibuted)                                                

TypeError: cannot parse from 'NoneType'

In [19]:
# fit model
history = model.fit(X, X, epochs=10, batch_size=5, verbose=2, validation_split=0.2)

Epoch 1/10


KeyboardInterrupt: 

In [ ]:
# plot training and validation loss
plt.plot(history.history['loss'], label='training loss')
plt.plot(history.history['val_loss'], label='validation loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# result analysis

In [16]:
from tabulate import tabulate

# model summary
model_summary = []
model.summary(print_fn=lambda x: model_summary.append(x))
model_summary = '\n'.join(model_summary)

# predicted and actual values
predicted = np.round(yhat, 3)
actual = np.round(X, 3)

# create table
table = [
    ['Model Summary', model_summary],
    ['Predicted Values', predicted],
    ['Actual Values', actual]
]

# convert table to LaTeX format
latex_table = tabulate(table, headers=['Variable', 'Value'], tablefmt='latex')

# print LaTeX table
print(latex_table)


\begin{tabular}{ll}
\hline
 Variable         & Value   \\
\hline
 Model Summary    & Model: "sequential"
\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_\_
 Layer (type)                Output Shape              Param \#   
 lstm (LSTM)                 (None, 10, 128)           154624    
                                                                 
 lstm\_1 (LSTM)               (None, 64)                49408     
                                                                 
 repeat\_vector (RepeatVector  (None, 10, 64)           0         
 )                                                               
                                                                 
 lstm\_2 (LSTM)               (None, 10, 64)            33024     
                                                                 
 lstm\_3 (LSTM)               (None, 10, 128)           98816     
                                 

In [20]:
# Reshape X and yhat to 2D arrays
X_2d = X.reshape(X.shape[0]*timesteps, n_features)
yhat_2d = yhat.reshape(X.shape[0]*timesteps, n_features)

In [21]:
# Calculate mean squared error
mse = mean_squared_error(X_2d, yhat_2d)
print("MSE: ", mse)

MSE:  0.0033856726


In [23]:
from sklearn.metrics import r2_score
r2 = r2_score(X_2d, yhat_2d)
print("R2 score: ", r2)

R2 score:  0.06690389884976325


In [41]:
np.argpartition(X[0][0],-1)[-1]

110

In [42]:
np.argpartition(yhat[0][0],-5)[-5:]

array([ 58,  68, 113,  32,  76])

In [58]:
#reconstruction cost for the test set


mse = np.mean(np.mean(np.power(X - yhat, 2), axis=1),axis=1)
mse

array([0.00485279, 0.00451607, 0.00409346, ..., 0.00446442, 0.00524236,
       0.00528777], dtype=float32)

# legacy

In [15]:
# fit model
model.fit(X, X, epochs=5, batch_size=5, verbose=2)
# demonstrate reconstruction
yhat = model.predict(X, verbose=0)
print('---Predicted---')
print(np.round(yhat,3))
print('---Actual---')
print(np.round(X, 3))

Epoch 1/5
998/998 - 41s - loss: 0.0049 - 41s/epoch - 41ms/step
Epoch 2/5
998/998 - 32s - loss: 0.0043 - 32s/epoch - 32ms/step
Epoch 3/5
998/998 - 32s - loss: 0.0039 - 32s/epoch - 32ms/step
Epoch 4/5
998/998 - 32s - loss: 0.0037 - 32s/epoch - 32ms/step
Epoch 5/5
998/998 - 33s - loss: 0.0035 - 33s/epoch - 33ms/step
---Predicted---
[[[-0.     0.005  0.008 ...  0.001  0.003  0.001]
  [-0.     0.007  0.014 ...  0.001  0.004  0.002]
  [ 0.     0.006  0.015 ...  0.     0.003  0.002]
  ...
  [ 0.007  0.006  0.006 ...  0.003 -0.001 -0.002]
  [ 0.006  0.011  0.008 ...  0.003  0.001 -0.003]
  [ 0.005  0.017  0.012 ...  0.004  0.004 -0.004]]

 [[ 0.     0.006  0.006 ...  0.001  0.003  0.   ]
  [ 0.     0.01   0.012 ...  0.001  0.004  0.002]
  [ 0.     0.01   0.015 ... -0.001  0.004  0.003]
  ...
  [ 0.005  0.01   0.01  ...  0.002  0.    -0.002]
  [ 0.006  0.013  0.011 ...  0.002  0.002 -0.002]
  [ 0.006  0.015  0.012 ...  0.002  0.004 -0.002]]

 [[ 0.001  0.008  0.008 ...  0.001  0.003  0.   ]
  [

In [26]:
#OLD


import json
import pandas as pd


# Load data from JSON file and Convert data to pandas DataFrame
df = pd.read_json('data.json', convert_dates=['timestamp'])


In [11]:
import keras

model = keras.Sequential()
model.add(keras.layers.LSTM(
    units=64,
    input_shape=(X.shape[1], X.shape[2])
))
model.add(keras.layers.Dropout(rate=0.2))
model.add(keras.layers.RepeatVector(n=X_train.shape[1]))
model.add(keras.layers.LSTM(units=64, return_sequences=True))
model.add(keras.layers.Dropout(rate=0.2))
model.add(
  keras.layers.TimeDistributed(
    keras.layers.Dense(units=X_train.shape[2])
  )
)

model.compile(loss='mae', optimizer='adam')

IndexError: tuple index out of range

In [5]:
#from sklearn.preprocessing import MinMaxScaler

# Convert categorical features to one-hot encoded format
#df = pd.get_dummies(df, columns=['requestUrl', 'status', 'source_app','requestMethod'])

# Normalize the numerical features
#scaler = MinMaxScaler()
#df[["latency", "requestSize", "responseSize"]] = scaler.fit_transform(df[["latency", "requestSize", "responseSize"]])
#df.columns.values.astype(list)

array(['latency', 'requestSize', 'requestMethod', 'responseSize',
       'requestUrl', 'status', 'source_app'], dtype=object)

In [16]:

test_array = np.array(test)
test_array = np.reshape(test_array, (test_array.shape[0], 1, test_array.shape[1]))

# Generate predictions on test data
y_pred = model.predict(test_array)

# Denormalize the predicted values
y_pred_denorm = scaler.inverse_transform(y_pred)

# Calculate the mean squared error
mse = np.mean(np.square(test - y_pred_denorm))

print('Test MSE:', mse)


8/8 [==============================] - 1s 3ms/step
Test MSE: latency                                                                                               0.011552
requestSize                                                                                           0.003098
responseSize                                                                                          0.053303
requestUrl_http://adservice:9555/hipstershop.AdService/GetAds                                         0.063806
requestUrl_http://cartservice:7070/hipstershop.CartService/AddItem                                    0.007903
requestUrl_http://cartservice:7070/hipstershop.CartService/EmptyCart                                  0.000008
requestUrl_http://cartservice:7070/hipstershop.CartService/GetCart                                    0.080683
requestUrl_http://checkoutservice:5050/hipstershop.CheckoutService/PlaceOrder                         0.000004
requestUrl_http://currencyservice:7000/hipstershop.

/opt/software/lib/python3.9/site-packages/numpy/core/fromnumeric.py:3438: FutureWarning: In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'
  return mean(axis=axis, dtype=dtype, out=out, **kwargs)
